In [37]:
import os
from os import listdir
import time
import gzip
import json


# Torch libs
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from torch.utils.data.sampler import WeightedRandomSampler

import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torchvision.io import read_image
import torch.optim as optim

from sklearn.model_selection import train_test_split

# Data libs
import pandas as pd
import numpy as np
# import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [38]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [39]:
os.chdir('data')

FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [40]:
df = getDF('Luxury_Beauty_5.json.gz')

In [41]:
#No. of reviews/counts per rating
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34278 entries, 0 to 34277
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         34278 non-null  float64
 1   verified        34278 non-null  bool   
 2   reviewTime      34278 non-null  object 
 3   reviewerID      34278 non-null  object 
 4   asin            34278 non-null  object 
 5   style           16841 non-null  object 
 6   reviewerName    34278 non-null  object 
 7   reviewText      34265 non-null  object 
 8   summary         34263 non-null  object 
 9   unixReviewTime  34278 non-null  int64  
 10  vote            6532 non-null   object 
 11  image           617 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 3.2+ MB


In [42]:
display(df.head(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 5, 2018",A2HOI48JK8838M,B00004U9V2,{'Size:': ' 0.9 oz.'},DB,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance,1515110400,NaN,NaN
1,5.0,True,"04 5, 2017",A1YIPEY7HX73S7,B00004U9V2,{'Size:': ' 3.5 oz.'},Ajaey,"wonderful hand lotion, for seriously dry skin,...",wonderful hand lotion,1491350400,NaN,NaN
2,5.0,True,"03 27, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 250 g'},D. Jones,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around,1490572800,NaN,NaN
3,5.0,True,"03 20, 2017",A2R4UNHFJBA6PY,B00004U9V2,{'Size:': ' 3.5 oz.'},Amazon Customer,Thanks!!,Five Stars,1489968000,NaN,NaN
4,5.0,True,"02 28, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 0.9 oz.'},D. Jones,Great hand lotion. Soaks right in and leaves ...,Great hand lotion!,1488240000,NaN,NaN
5,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 250 g'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
6,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 3.5 oz.'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
7,5.0,True,"01 30, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 0.9 oz.'},Amr,Just as described. Arrived on time.,Five Stars,1485734400,NaN,NaN
8,4.0,False,"01 24, 2017",A1YY53NQXFKMRN,B00004U9V2,{'Size:': ' 3.5 oz.'},Trixie,Nice lightweight hand cream for the summer.,"Smells good, absorbs quickly",1485216000,NaN,NaN
9,5.0,True,"12 1, 2016",A3R0NQ9E53JHYQ,B00004U9V2,{'Size:': ' 250 g'},T. Hooth,Best hand cream ever.,Five Stars,1480550400,NaN,NaN


In [43]:
display(df.tail(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
34268,5.0,True,"08 21, 2018",A1ID5QT86H8J9D,B01GOZ61O8,NaN,JSM,Love it. Just wish it came it bigger tube. I...,Great SPF!,1534809600,NaN,NaN
34269,4.0,False,"05 21, 2018",A1G60ANYI0HMEU,B01GOZ61O8,NaN,Jeddy 3,"I really like the Tizo sunscreens, but this on...","Not sure what else it's doing, but it's defini...",1526860800,NaN,NaN
34270,3.0,False,"12 10, 2017",ATN4J5VS8Q0YM,B01GOZ61O8,NaN,Anna Hope,I used this on a recent trip across the countr...,Not Everyday,1512864000,NaN,NaN
34271,4.0,False,"10 10, 2017",AVL5IDEE8YA3D,B01GOZ61O8,NaN,Rachel McElhany,This product doesn't really say what it is. Th...,TIZO AM Replenish,1507593600,NaN,NaN
34272,2.0,False,"09 9, 2017",A2X2WTEVCZ5L8N,B01GOZ61O8,NaN,Sandy Kay,I'm not sure whether it is the ceramicides or ...,Hated how it felt on my face,1504915200,NaN,NaN
34273,4.0,False,"09 3, 2017",A2CF66KIQ3RKX3,B01GOZ61O8,NaN,Vivian Deliz,I like to use moisturizers and sunscreens that...,Works great as a moisturizer and sunscreen,1504396800,NaN,NaN
34274,4.0,False,"09 3, 2017",A1LKOIZXPQ9VG0,B01GOZ61O8,NaN,Elisa 20,I wouldn't be able to afford this if not asked...,Nice skin care product and sunscreen if you do...,1504396800,NaN,NaN
34275,1.0,True,"08 25, 2017",AV2RWORXTFRJU,B01H353HUY,NaN,Gapeachmama,Did nothing,One Star,1503619200,NaN,NaN
34276,5.0,False,"07 8, 2017",A22S7D0LP8GRDH,B01H353HUY,NaN,Jacob and Kiki Hantla,I love the Oribe bright blonde radiance spray....,No more brass!,1499472000,NaN,NaN
34277,5.0,True,"07 9, 2018",AAF5D1LTFGB7L,B01HGSJPMW,NaN,Libby Johnson,I love all of the Elemis products.,Five Stars,1531094400,NaN,NaN


In [44]:
#Number of records
print(len(df.index))

34278


In [45]:
#No. of reviews/counts per rating
df.groupby(by="overall").size()

overall
1.0     1095
2.0     1496
3.0     3884
4.0     7833
5.0    19970
dtype: int64

In [46]:
#df.groupby(by="reviewText").size()

In [47]:
df.groupby(by="reviewerName").size()

reviewerName
 B Mouk              8
 Catrinka            6
 Daba                1
 Indiangirl          7
 JOKE                4
                    ..
yvonne hankins       8
zenigma283          11
~*~ Sunshine ~*~    16
~Kimber~             8
~purplemoon~         7
Length: 3584, dtype: int64

In [48]:
#calculate average length (mean) of reviewText
df['reviewText'].str.len().mean()

489.144024514811

In [49]:
#calculate average length (mean) of reviewText
df['overall'].mean()

4.286160219382695

In [50]:
df.describe()

,overall,unixReviewTime
count,34278.00000,3.427800e+04
mean,4.28616,1.445171e+09
std,1.03736,4.962381e+07
min,1.00000,1.127174e+09
25%,4.00000,1.411430e+09
50%,5.00000,1.448410e+09
75%,5.00000,1.484093e+09
max,5.00000,1.537747e+09


In [51]:
#Create Distribution graph

In [52]:
# Create new column with precip in the original units (inches)
#df["averageRating"] = df["overall"].mean()

#df

In [53]:
#No. of reviews/counts per rating 
df1=df[['reviewTime', 'overall']] 
df1
 

,reviewTime,overall
0,"01 5, 2018",5.0
1,"04 5, 2017",5.0
2,"03 27, 2017",5.0
3,"03 20, 2017",5.0
4,"02 28, 2017",5.0
...,...,...
34273,"09 3, 2017",4.0
34274,"09 3, 2017",4.0
34275,"08 25, 2017",1.0
34276,"07 8, 2017",5.0


In [54]:
df1[['reviewTime', 'year']]=df1['reviewTime'].str.split(',', 1, expand=True) 
df1

#df[['team', 'conference']] = df['team'].str.split(',', 1, expand=True)

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,reviewTime,overall,year
0,01 5,5.0,2018
1,04 5,5.0,2017
2,03 27,5.0,2017
3,03 20,5.0,2017
4,02 28,5.0,2017
...,...,...,...
34273,09 3,4.0,2017
34274,09 3,4.0,2017
34275,08 25,1.0,2017
34276,07 8,5.0,2017


In [55]:
df2=df1[['year','overall']]
df2.rename(columns={'overall':'rating'}, inplace=True)
df2 

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year,rating
0,2018,5.0
1,2017,5.0
2,2017,5.0
3,2017,5.0
4,2017,5.0
...,...,...
34273,2017,4.0
34274,2017,4.0
34275,2017,1.0
34276,2017,5.0


In [56]:
#Calculate the counts group by year and overall
df2 = df2.value_counts().reset_index(name='counts')  

In [57]:
#Sort in descending order
df3=df2.sort_values(by=['year','rating'], ascending=False)
df3

,year,rating,counts
9,2018,5.0,1042
14,2018,4.0,514
21,2018,3.0,273
30,2018,2.0,99
32,2018,1.0,86
3,2017,5.0,3571
6,2017,4.0,1803
10,2017,3.0,880
18,2017,2.0,310
25,2017,1.0,188


In [58]:
fig = px.bar(df3, x="year", y="counts", color = 'rating', title="Year-wise number of ratings")
fig.show()